In [1]:
from dotenv import load_dotenv
import os
import requests
import alpaca_trade_api as tradeapi
import pandas as pd
import numpy as np

In [2]:
load_dotenv('api.env')

True

In [3]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [4]:
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [9]:
alpaca = tradeapi.REST(
    alpaca_api_key,alpaca_secret_key,
    api_version="v2")

In [10]:
symbol='SPY'
timeframe = '1H'
start_date = pd.Timestamp("2013-01-01", tz="America/New_York").isoformat()

In [69]:
from finta import TA
historical_data = alpaca.get_bars(symbol, timeframe, start=start_date).df
historical_data['delta']=historical_data['close'].diff()

In [70]:
historical_data

,open,high,low,close,volume,trade_count,vwap,delta
timestamp,,,,,,,,
2015-12-01 09:00:00+00:00,209.43,209.440,209.2400,209.30,30375,82,209.322606,NaN
2015-12-01 10:00:00+00:00,209.30,209.360,209.0700,209.36,54490,118,209.214253,0.06
2015-12-01 11:00:00+00:00,209.37,209.650,209.3700,209.44,44404,109,209.526441,0.08
2015-12-01 12:00:00+00:00,209.45,209.590,209.3300,209.51,111106,234,209.460429,0.07
2015-12-01 13:00:00+00:00,209.52,209.670,209.2700,209.32,426233,645,209.429479,-0.19
...,...,...,...,...,...,...,...,...
2023-03-27 17:00:00+00:00,396.45,397.545,396.0400,396.83,6612821,61078,396.892699,0.40
2023-03-27 18:00:00+00:00,396.84,398.120,396.3850,398.04,6596038,57357,397.311483,1.21
2023-03-27 19:00:00+00:00,398.04,398.270,395.8400,396.45,18722834,131612,396.769091,-1.59


**VWAP Trading Signal**

We decide to buy/sell at points where the market price crosses the vwap.

In [73]:
historical_data['Predicted-vwap']=np.where(historical_data['vwap']>=historical_data['close'], 1,0)
historical_data['vwap-entry/exit']=historical_data['Predicted-vwap'].diff()
historical_data


,open,high,low,close,volume,trade_count,vwap,delta,Predicted-vwap,vwap-entry/exit,rsi
timestamp,,,,,,,,,,,
2015-12-01 09:00:00+00:00,209.43,209.440,209.2400,209.30,30375,82,209.322606,NaN,1,NaN,64.376590
2015-12-01 10:00:00+00:00,209.30,209.360,209.0700,209.36,54490,118,209.214253,0.06,0,-1.0,64.376590
2015-12-01 11:00:00+00:00,209.37,209.650,209.3700,209.44,44404,109,209.526441,0.08,1,1.0,64.376590
2015-12-01 12:00:00+00:00,209.45,209.590,209.3300,209.51,111106,234,209.460429,0.07,0,-1.0,64.376590
2015-12-01 13:00:00+00:00,209.52,209.670,209.2700,209.32,426233,645,209.429479,-0.19,1,1.0,64.376590
...,...,...,...,...,...,...,...,...,...,...,...
2023-03-27 17:00:00+00:00,396.45,397.545,396.0400,396.83,6612821,61078,396.892699,0.40,1,1.0,48.305458
2023-03-27 18:00:00+00:00,396.84,398.120,396.3850,398.04,6596038,57357,397.311483,1.21,0,-1.0,44.191019
2023-03-27 19:00:00+00:00,398.04,398.270,395.8400,396.45,18722834,131612,396.769091,-1.59,1,1.0,46.903255


In [74]:
from functools import lru_cache
@lru_cache(maxsize=None)

SyntaxError: unexpected EOF while parsing (4250197568.py, line 2)

**RSI**

The stock is considered overbought when the RSI rises above 70 and considered oversold when the RSI dips below 30 

In [ ]:
import numpy as np

def calculate_rsi(price_diff, n=14):
    rsi=[0]*len(price_diff)
    #first n days
    up_mean=price_diff.head(n)[price_diff>0].mean(skipna=True)
    down_mean=abs(price_diff.head(n)[price_diff<=0].mean(skipna=True))
    rs=float(up_mean/down_mean)
    rsi[:n]=[100-(100/(1+rs))]*n

    for i in range(n,len(price_diff)):
        up_mean=price_diff[i-n:i][price_diff>0].mean()
        down_mean=abs(price_diff[i-n:i][price_diff<=0]).mean(skipna=True)
        rs=float(up_mean/down_mean)
        rsi[i]=100-(100/(1+rs))
    return rsi
historical_data['rsi'] = calculate_rsi(historical_data.loc[:,['delta']])




In [85]:
def sma(n)

sma10=TA.SMA(historical_data,period=10)
historical_data['sma-10']=sma10
sma=[0]*len(historical_data['close'])
sum=0
close=historical_data['close']
for i in range(10):
    sum+=close[i]
avg=sum/10
historical_data.loc[historical_data.index[:10], 'sma-10'] = avg

sma50=TA.SMA(historical_data,period=50)
historical_data['sma-50']=sma50
sma=[0]*len(historical_data['close'])
sum=0
close=historical_data['close']
for i in range(50):
    sum+=close[i]
avg=sum/50
historical_data.loc[historical_data.index[:50], 'sma-50'] = avg

sma50=TA.SMA(historical_data,period=50)
historical_data['sma-50']=sma50
sma=[0]*len(historical_data['close'])
sum=0
close=historical_data['close']
for i in range(50):
    sum+=close[i]
avg=sum/50
historical_data.loc[historical_data.index[:50], 'sma-50'] = avg

historical_data

,open,high,low,close,volume,trade_count,vwap,delta,Predicted-vwap,vwap-entry/exit,...,Predicted ema-3,ema-3-entry/exit,Predicted ema-5,ema-5-entry/exit,Predicted ema-8,ema-8-entry/exit,Predicted ema-13,ema-13-entry/exit,sma-10,sma-50
timestamp,,,,,,,,,,,,,,,,,,,,,
2015-12-01 09:00:00+00:00,209.43,209.440,209.2400,209.30,30375,82,209.322606,NaN,1,NaN,...,0,NaN,0,NaN,0,NaN,0,NaN,209.65000,209.028698
2015-12-01 10:00:00+00:00,209.30,209.360,209.0700,209.36,54490,118,209.214253,0.06,0,-1.0,...,1,1.0,1,1.0,1,1.0,1,1.0,209.65000,209.028698
2015-12-01 11:00:00+00:00,209.37,209.650,209.3700,209.44,44404,109,209.526441,0.08,1,1.0,...,1,0.0,1,0.0,1,0.0,1,0.0,209.65000,209.028698
2015-12-01 12:00:00+00:00,209.45,209.590,209.3300,209.51,111106,234,209.460429,0.07,0,-1.0,...,1,0.0,1,0.0,1,0.0,1,0.0,209.65000,209.028698
2015-12-01 13:00:00+00:00,209.52,209.670,209.2700,209.32,426233,645,209.429479,-0.19,1,1.0,...,0,-1.0,0,-1.0,0,-1.0,0,-1.0,209.65000,209.028698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-27 17:00:00+00:00,396.45,397.545,396.0400,396.83,6612821,61078,396.892699,0.40,1,1.0,...,1,1.0,1,1.0,1,1.0,1,1.0,397.20399,394.742196
2023-03-27 18:00:00+00:00,396.84,398.120,396.3850,398.04,6596038,57357,397.311483,1.21,0,-1.0,...,1,0.0,1,0.0,1,0.0,1,0.0,397.32499,394.732196
2023-03-27 19:00:00+00:00,398.04,398.270,395.8400,396.45,18722834,131612,396.769091,-1.59,1,1.0,...,0,-1.0,0,-1.0,0,-1.0,0,-1.0,397.24999,394.689996


We can use RSI in conjunction with the moving average---where we buy when the RSI is below 30 and the price crosses above the moving average and we sell when the RSI is above 70 and the price crosses below the moving average 

In [ ]:
historical_data['rsi predictions']=np.where(historical_data['rsi']>30 and ,)

In [15]:
historical_data

,open,high,low,close,volume,trade_count,vwap,delta,Predicted-vwap,entry/exit,rsi
timestamp,,,,,,,,,,,
2015-12-01 09:00:00+00:00,209.43,209.440,209.240,209.30,30375,82,209.322606,NaN,1,NaN,64.376590
2015-12-01 10:00:00+00:00,209.30,209.360,209.070,209.36,54490,118,209.214253,0.06,0,-1.0,64.376590
2015-12-01 11:00:00+00:00,209.37,209.650,209.370,209.44,44404,109,209.526441,0.08,1,1.0,64.376590
2015-12-01 12:00:00+00:00,209.45,209.590,209.330,209.51,111106,234,209.460429,0.07,0,-1.0,64.376590
2015-12-01 13:00:00+00:00,209.52,209.670,209.270,209.32,426233,645,209.429479,-0.19,1,1.0,64.376590
...,...,...,...,...,...,...,...,...,...,...,...
2023-03-27 15:00:00+00:00,396.41,396.870,395.560,395.96,6734496,67498,396.317212,-0.45,1,0.0,49.343340
2023-03-27 16:00:00+00:00,395.95,396.970,395.570,396.43,6715820,61840,396.362386,0.47,0,-1.0,51.631834
2023-03-27 17:00:00+00:00,396.45,397.545,396.040,396.83,6612821,61078,396.892699,0.40,1,1.0,48.305458


In [82]:
ema3 = TA.EMA(historical_data, period=3)
ema5 = TA.EMA(historical_data, period=5)
ema8 = TA.EMA(historical_data,period=8)
ema13 = TA.EMA(historical_data,period=13)
vwap=TA.VWAP(historical_data)


historical_data['ema-3']=ema3
historical_data['ema-5']=ema5
historical_data['ema-8']=ema8
historical_data['ema-13']=ema13

historical_data['Predicted ema-3']=np.where(historical_data['ema-3'].diff()>0, 1,0)
historical_data['ema-3-entry/exit']=historical_data['Predicted ema-3'].diff()

historical_data['Predicted ema-5']=np.where(historical_data['ema-5'].diff()>0, 1,0)
historical_data['ema-5-entry/exit']=historical_data['Predicted ema-5'].diff()


historical_data['Predicted ema-8']=np.where(historical_data['ema-8'].diff()>0, 1,0)
historical_data['ema-8-entry/exit']=historical_data['Predicted ema-8'].diff()

historical_data['Predicted ema-13']=np.where(historical_data['ema-13'].diff()>0, 1,0)
historical_data['ema-13-entry/exit']=historical_data['Predicted ema-13'].diff()



#just to confirm that this is accurate and matches with the vwap from alpaca


In [83]:
historical_data

,open,high,low,close,volume,trade_count,vwap,delta,Predicted-vwap,vwap-entry/exit,...,ema-8,ema-13,Predicted ema-3,ema-3-entry/exit,Predicted ema-5,ema-5-entry/exit,Predicted ema-8,ema-8-entry/exit,Predicted ema-13,ema-13-entry/exit
timestamp,,,,,,,,,,,,,,,,,,,,,
2015-12-01 09:00:00+00:00,209.43,209.440,209.2400,209.30,30375,82,209.322606,NaN,1,NaN,...,209.300000,209.300000,0,NaN,0,NaN,0,NaN,0,NaN
2015-12-01 10:00:00+00:00,209.30,209.360,209.0700,209.36,54490,118,209.214253,0.06,0,-1.0,...,209.333750,209.332308,1,1.0,1,1.0,1,1.0,1,1.0
2015-12-01 11:00:00+00:00,209.37,209.650,209.3700,209.44,44404,109,209.526441,0.08,1,1.0,...,209.378342,209.373858,1,0.0,1,0.0,1,0.0,1,0.0
2015-12-01 12:00:00+00:00,209.45,209.590,209.3300,209.51,111106,234,209.460429,0.07,0,-1.0,...,209.424486,209.416118,1,0.0,1,0.0,1,0.0,1,0.0
2015-12-01 13:00:00+00:00,209.52,209.670,209.2700,209.32,426233,645,209.429479,-0.19,1,1.0,...,209.392028,209.390564,0,-1.0,0,-1.0,0,-1.0,0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-27 17:00:00+00:00,396.45,397.545,396.0400,396.83,6612821,61078,396.892699,0.40,1,1.0,...,396.787631,396.538305,1,1.0,1,1.0,1,1.0,1,1.0
2023-03-27 18:00:00+00:00,396.84,398.120,396.3850,398.04,6596038,57357,397.311483,1.21,0,-1.0,...,397.065936,396.752833,1,0.0,1,0.0,1,0.0,1,0.0
2023-03-27 19:00:00+00:00,398.04,398.270,395.8400,396.45,18722834,131612,396.769091,-1.59,1,1.0,...,396.929061,396.709571,0,-1.0,0,-1.0,0,-1.0,0,-1.0


In [55]:
dir([TA.__class__])

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']